# TP4 : Théorie de l'information

L'objectif de ce TP est d'illustrer en particulier la sous-section 7.6.4 "Pourquoi utiliser un dictionnaire ?" du polycopié.

In [1]:
# Imports classiques
import numpy as np
import matplotlib.pyplot as plt

On considère une source connue, markovienne en régime stationnaire, qui produit des messages de longueur $N$ fixée. On veut coder ces messages en mots de code composés de bits 0 ou 1, de sorte que ces messages codés (non cryptés) soient de longueur la plus courte possible en moyenne.

Cela signifie que les messages sont en fait les trajectoires finies $X_1\ldots X_N$ d'une chaîne de markov (concaténées en un mot), d'espace d'état un alphabet connu, de matrice de transition connue, et de probabilité initiale invariante connue.

On pourra choisir pour ce TP d'utiliser les valeurs suivantes : $N=30$, l'espace d'état de la chaîne de Markov est l'alphabet $\{A, B, C\}$, la matrice de transition est
$$ P=\begin{pmatrix} 0 & \frac12 & \frac12 \\ \frac14 & \frac12 & \frac14 \\ \frac14 & \frac14 & \frac12  \end{pmatrix}, $$
dont on a déjà calculé la probabilité invariante
$$ \pi = (1/5, 2/5, 2/5). $$

Des messages pourront être simulés en utilisant votre code du TP3. Si besoin, on pourrat également vérifier que les algorithmes fonctionnent sur le message suivant :
$$ CCACCBBACACBBBCCCABCACBBBACBBB $$

In [2]:
N=30
P=np.array([[0,.5,.5],[.25,.5,.25],[.25,.25,.5]])
pinv=np.array([1/5,2/5,2/5])
alph="ABC"
message = "CCACCBBACACBBBCCCABCACBBBACBBB"

## Exercice 1

### Question 1

Sur brouillon, indentifier un code de Huffman sur l'alphabet alph (le dictionnaire est l'alphabet), associé à la probabilité de codage pinv. Cela pourra se faire au moyen d'un arbre binaire (voir section 7.3 du cours, et 7.5 pour la définition du code de Huffman)

On peut par exemple coder $B$ par 1, $C$ par 00 et $A$ par 01.

### Question 2

Écrire deux fonctions codeur_Huffman_unitaire() et decodeur_Huffman_unitaire(), qui respectivement encode un unique symbole de l'alphabet selon le code de Huffamn de la question précédente, et décode le mot de code produit.

In [3]:
def codeur_Huffman_unitaire(symbole):
    if symbole == 'B':
        return '1'
    elif symbole == 'A':
        return '01'
    elif symbole == 'C':
        return '00'

def decodeur_Huffman_unitaire(chaine):
    if chaine[0] == '1':
        return 'B'
    elif chaine[1] == '1':
        return 'A'
    elif chaine[1] == '0':
        return 'C'

### Question 3

Écrire une fonction codeur_Huffman() qui encode successivement les symboles d'un message (mot) et concatène les mots de codes obtenus en un seul message codé. Écrire sa réciproque decodeur_Huffman(). Tester votre production !

In [4]:
def codeur_Huffman(mot):
    code = ''
    for c in mot:
        code += codeur_Huffman_unitaire(c)
    return code

def decodeur_Huffman(code):
    mot = ''
    l = len(code)
    i = 0
    while i < l:
        chaine = code[i:]
        if chaine[0] == '1':
            mot += 'B'
            i += 1
        elif chaine[1] == '1':
            mot += 'A'
            i += 2
        elif chaine[1] == '0':
            mot += 'C'
            i += 2
    return mot

In [5]:
mot = 'ABCBACBBCABBCABCBABCBA'
print(codeur_Huffman(mot))
print(decodeur_Huffman(codeur_Huffman(mot)))
print(mot)

0110010100110001110001100101100101
ABCBACBBCABBCABCBABCBA
ABCBACBBCABBCABCBABCBA


## Exercice 2 : Code arithmétique

### Question 0 facultative, à faire à la fin s'il reste du temps

Écrire une fonction check_parameters() qui prend en entrée un message (m, de type string), un alphabet (a, de type string), une matrice de transition (Q, de type np.ndarray) et une probabilité invariante (qinv, de type np.ndarray). La fonction doit vérifier
- les types des arguments,
- que l'alphabet ne contient qu'un fois chaque symbole,
- que tous les symboles du message sont dans l'aphabet,
- que ni le message ni l'alphabet ne sont vides,
- que la matrice est bien une matrice de transition (y compris la positivité de toutes les cases),
- que la proba est bien une proba, et est bien invariante.

Si tout va bien, la fonction check_parameters() doit renvoyer la valeur entière 0, sinon toute autre valeur entière strictement positive de votre choix.

In [6]:
# à compléter
def check_parameters(m, a, Q, qinv):
    if type(m) != str or type(a) != str or type(Q) != np.ndarray or type(qinv) != np.ndarray:
        return 1       # Problème de type dans les arguments
    
    for i in range(len(a)):
        if a[i] in a[i+1:]:
            return 2   #lettres non uniques dans l'alphabet
    
    for c in m:
        if c not in a:
            return 3   #lettres du messages pas dans l'alphabet
    
    if a == "" or m == "":
        return 4       # alphabet ou message vide
    
    if not np.all(Q >= 0) or not np.all(np.sum(Q, axis = 1) == 1):
        return 5       # pas une matrice de transition
    
    if not np.all(qinv >= 0) or np.sum(qinv) != 1 or not np.all(np.dot(qinv,Q) == qinv):
        return 6       # pas une mesure de proba invariante    
    
    return 0


In [7]:
check_parameters(message, alph, P, pinv)

0

Voici une description du codage arithmétique dans le cas d'un message issu d'une chaîne de Markov (librement adapté de http://www.ittc.ku.edu/~jsv/Papers/HoV94.arithmetic_codingOfficial.pdf, partie II - A. Vous trouverez des schémas explicatifs dans l'article).

L'algorithme pour coder un message fonctionne de la manière suivante:
1. On commence avec un intervalle $[\text{début}, \text{début} + \text{précision}[$ initialisé à $[0,1[$
2. Pour chaque lettre du message, on effectue deux étapes:
    1. On divise l'intervalle actuel en sous-intervalles, un pour chaque lettre contenue dans l'alphabet. La taille de ces sous-intervalles est proportionnelle à la probabilité d'apparition de la lettre correspondante.
    2. On sélectionne l'intervalle correspondant à la lettre lue, et on en fait le nouvel intervalle: on actualise les variables début et précision.
3. On renvoie le point milieu du dernier intervalle avec un nombre de bits suffisant pour le distinguer des autres intervalles finaux possibles.

Dans notre cas, le message est créé à partir d'une chaîne de Markov initialisée à la mesure invariante:
- Pour le premier caractère, la probabilité d'apparition de chaque lettre correspond au coefficient associé dans la probabilité invariante.
- Pour les caractères suivants, la probabilité d'apparition de chaque lettre est la probabilité de transition de la lettre précédente à celle-ci.

La longueur de l'intervalle final (précision) est donc le produit des probabilités d'apparition de chaque lettre dans le message. L'étape finale utilise au plus $-\lfloor \log_2 p \rfloor + 2$ bits pour distinguer le message codé des autres messages de même longueur.

Dans l'étape 2, on a seulement besoin de calculer le sous-intervalle correspondant à la lettre $a_i$ qui apparaît. Pour cela, on peut utiliser la probabilité cumulée (avec $P(a_i) = p_i$, soit un coefficent de $Q$ soit de $q_{inv}$)
$$
P_C = \sum_{k=1}^{i-1}p_k
$$
et la suivante 
$$
P_N = P_C + p_i = \sum_{k=1}^{i}p_k.
$$

Le nouveau sous-intervalle sera alors: $$[\text{début} + P_C \cdot \text{précision} ; \text{début} + P_N \cdot \text{précision}]$$. Les nouvelles valeurs de début et précision seront donc:
$$
\text{début} = \text{début} + P_C \cdot \text{précision} \\
\text{précision} = \text{précision}\cdot p_i
$$

### Question 1

Lire attentivement le code suivant. Identifiez-vous chaque étape ?

In [8]:
def codeur_arithmetique(message, a, Q, qinv):
    if check_parameters(message, a, Q, qinv)!=0 :
        raise Exception("Bah alors, qu'avez-vous foutu ?")
    # paramètres auxiliaires
    cumQ = np.cumsum(Q,axis=1)
    cumqinv=np.cumsum(qinv)
    # Initialisation
    lasti=-1
    # règle de chaînage
    for c in message:
        i = a.find(c)
        if lasti==-1:
            debut = cumqinv[i] - qinv[i]
            precision = qinv[i]
        else:
            debut = debut + precision*(cumQ[lasti,i] - Q[lasti,i])
            precision = precision*Q[lasti,i]
        lasti = i
    # longueur de code
    l = np.floor(- np.log2(precision)+2)
    # nombre de code
    res = int(np.floor(2**l*(debut+precision/2)))
    # conversion en binaire
    return np.binary_repr(res, width = int(l))


In [9]:
#codeur_arithmetique(message, alph, P, pinv)
codeur_arithmetique(message,alph,P,pinv)

'11010111011101010111000111000010001010101100'

### Question 2

Écrire la fonction decodeur_arithmetique(). Ses arguments pourront être 
* la longueur n du message avant codage (pour vous faciliter la vie, vous pourriez vous en passer),
* le message codé au format renvoyé par codeur_arithmetique(),
* l'alphapet
* la matrice de transition
* la probabilité invariante

La fonction decodeur_arithmetique() doit correctement renvoyer le message initial décodé, au format initial (string).

In [10]:
# à compléter
def decodeur_arithmetique(n, mcode, a, Q, qinv):
    #parametres auxiliaires
    cumQ = np.cumsum(Q,axis=1)
    cumqinv=np.cumsum(qinv)    
    #conversion en entier
    l = len(mcode)
    mcode = int(mcode,2)
    message = ""
    #initialisation
    lasti = -1
    debut = mcode*2**(-l)
    for j in range(n):
        i = 0
        if lasti == -1:
            while cumqinv[i]<debut:
                i+=1
            debut = (debut - cumqinv[i] + qinv[i])/qinv[i]
        else:
            while cumQ[lasti,i]<debut:
                i+=1
            debut = (debut - cumQ[lasti,i] + Q[lasti,i])/Q[lasti,i]
        message += a[i]
        lasti = i
    return message

In [11]:
# Tester votre fonction
message = "CCACCBBACACBBBCCCABCACBBBACBBB"
messageCode = codeur_arithmetique(message, alph, P, pinv)
messageDecode = decodeur_arithmetique(len(message), messageCode, alph, P, pinv)
# écrire un test ?
print(messageCode)
print(message)
print(messageDecode)
print(message == messageDecode)

11010111011101010111000111000010001010101100
CCACCBBACACBBBCCCABCACBBBACBBB
CCACCBBACACBBBCCCABCACBBBACBBB
True


### Question 3 

Comparez les longueurs de code obtenues dans les deux exercices

In [12]:
print(codeur_arithmetique(message, alph, P, pinv))
print(codeur_Huffman(message))

11010111011101010111000111000010001010101100
000001000011010001001110000000110001001110100111


### Question 4 facultative

Calculer la différence moyenne attendue entre les longueurs de code des deux exercices.

Indication: le poly (sous-section 7.6.4) donne un différence moyenne approximative $I(X_1; \ldots; X_n)$ (à affiner en calculant $\delta$), et l'exercice 7.4 améliore la formule.

## Exercice 3

*Amusez-vous !*

Pistes~:
- comparer les longueurs de code obtenues dans les deux exercices sur différents messages issus de la même source (une procédure d'estimation par Monte-Carlo est envisageable) ;
- essayer avec différentes sources ;
- faire la question 0 de l'exercice 1 ;
- etc.